In [1]:
from dataclasses import dataclass
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser

%load_ext sql

Deploy Streamlit apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [ ]:
config = ConfigParser()
config.read("../.config")

In [ ]:
API_KEY = config['GCP']['API_KEY']
radius = 1500  # in meters
place_type = 'shopping'  # Example place type
address = 'Luxembourg City, Luxembourg'


In [ ]:
def format_coordinates(latitude, longitude):
    return f"{latitude},{longitude}"

def get_coordinates(api_key, address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error in response: {data['status']}")
            return None, None
    else:
        print(f"HTTP error: {response.status_code}")
        return None, None


latitude, longitude = get_coordinates(API_KEY, address)

if latitude and longitude:
    formatted_location = format_coordinates(latitude, longitude)
    # print(formatted_location)  # Output will be the coordinates of Luxembourg City
else:
    print("Could not retrieve coordinates.")


In [ ]:
# url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&type={place_type}&key={API_KEY}"

response = requests.get(url)
places = response.json()

for place in places['results']:
    print(f"Name: {place['name']}")
    print(f"Address: {place.get('vicinity', 'N/A')}")
    print(f"Place ID: {place['place_id']}")
    print('---')


In [ ]:
places['results'][0]

In [ ]:
place_id = 'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'  # Example place ID

details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

details_response = requests.get(details_url)
place_details = details_response.json()


print(f"Name: {place_details['result']['name']}")
print(f"Address: {place_details['result']['formatted_address']}")
print(f"Phone Number: {place_details['result'].get('formatted_phone_number', 'N/A')}")
print(f"Website: {place_details['result'].get('website', 'N/A')}")


In [ ]:
place_details['result']

In [7]:
# Connect to DuckDB
# con = duckdb.connect('places.db')
# df = con.execute('SELECT * FROM places').fetchdf()

In [12]:
conn = duckdb.connect('places.db')
%sql conn --alias duckdb

In [16]:
%%sql
select * from  places;


Running query in 'duckdb'

place_id,name,vicinity,latitude,longitude
ChIJtZdt6IJVlUcRgHIF0dbRAAQ,Ettelbruck,Ettelbruck,49.8442568,6.0984783
ChIJtdx-iY39v0cRtMZ0E03pqw0,Hotel du Parc,"28 Avenue de la Gare, Diekirch",49.8659659,6.158462
ChIJ4-vpDmZRlUcRFfOuBiVpwU4,Hostellerie Val Fleuri,"28 Rue Lohr, Mersch",49.750239,6.114546000000001
ChIJm-fLYWJRlUcRGxBrch-VVA8,Institute National Des Langues Luxembourg - Site Mersch,"57 Rue Grande-Duchesse Charlotte, Mersch",49.74798329999999,6.104214599999999
ChIJASJryvpWlUcRkVGQwjsSb3E,Epicerie am Duerf,17 Rue Principale,49.8045701,6.154338300000001
ChIJI3SHyuBZlUcR_PeU10XZHXE,Youth Hostel Larochette,"45 Rue Osterbour, Larochette",49.789296,6.217893000000001
ChIJ-Wcy7ZP9v0cRGbJr1d4ifpA,Restaurant La Fondue,"12 Esplanade, Diekirch",49.86997659999999,6.159214599999999
ChIJ-5tgrVpVlUcRDlMJIB-3oY8,Oa6 Centre de Santé et de Bien-Etre,"B, 13, Route d'Arlon",49.8470581,6.033528599999999
ChIJBdMVYVxRlUcRIj_mKYeA9jg,Hôtel Martha Mersch Luxembourg,2 Am Kaesch,49.7593891,6.1147671
ChIJ68K9bx3-v0cR7StEVFXoIgA,Hôtel-Restaurant Dahm,"Sûre, sur, 57 Porte des Ardennes, Erpeldange-sur-Sûre",49.8653099,6.113130399999998


In [11]:
# con.close()
conn.close()

In [ ]:
def get_cities_towns(api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address=Luxembourg&key={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if data['status'] == 'OK':
        results = data['results']
        places = []
        for result in results:
            places.append(result['formatted_address'])
        return places
    else:
        print(f"Error: {data['status']}")


cities_towns = get_cities_towns(API_KEY)
print(cities_towns)
